In [ ]:
import pandas as pd

In [ ]:
pd.__version__

In [ ]:
crd = "D:\\Projects\\mystoreapp\\data\\CreditDB.csv"
pmt = "D:\\Projects\\mystoreapp\\data\\Payments.csv"

dfpay = pd.read_csv(pmt, parse_dates=['Date'], dayfirst=True)
dfcrd = pd.read_csv(crd, parse_dates=['Date'], dayfirst=True)

In [ ]:
df = pd.concat([dfpay, dfcrd])

In [ ]:
df.sort_values(by=['Customer Name', 'Date'], inplace=True, ascending=True)

In [ ]:
df.tail(15)

In [ ]:
look_up = {'01': 'Jan', '02': 'Feb', '03': 'Mar', '04': 'Apr', '05': 'May',
            '06': 'Jun', '07': 'Jul', '08': 'Aug', '09': 'Sep', '10': 'Oct', '11': 'Nov', '12': 'Dec'}

In [ ]:
df['Year'] = pd.DatetimeIndex(df['Date']).year
df['Month'] = pd.DatetimeIndex(df['Date']).month
df['Month'] = df['Month'].apply(lambda x: look_up[x])
df['Customer Name'] = df['Customer Name']+' ('+df['Customer Code'].astype(str)+')'

In [ ]:
df = df.groupby(['Customer Name', 'Year', 'Month'])[['Bill Total', 'Amount Paid']].agg('sum').reset_index()

In [ ]:
df.sort_values(by=['Customer Name', 'Year', 'Month'], inplace=True, ascending=False)

### Reports

In [17]:
import pandas as pd
from datetime import datetime
import numpy as np

In [18]:
Path = "D:\\Projects\\mystoreapp\\"

In [19]:
bv = pd.read_csv(Path+"data/CreditDB.csv", parse_dates=['Date']).dropna()
pv = pd.read_csv(Path+"data/Payments.csv", parse_dates=['Date']).dropna()

In [20]:
print(bv.shape)
bv.head(2)

(4168, 6)


Bill No Customer Name  Customer Code                Date  Bill Total  \
0        1      ABHAYRAJ             59 2021-11-22 16:53:00      1861.0   
1        2      ABHAYRAJ             59 2021-11-29 10:20:00       398.0   

  Payment  
0  CREDIT  
1  CREDIT

In [21]:
print(pv.shape)
pv.head(2)

(153, 5)


Customer Name  Customer Code                Date  Amount Paid    Received By
0         ASHOK             39 2021-05-08 21:11:00         4000  KRISHNA STORE
1        RAJANI             58 2021-11-25 17:28:00          400  KRISHNA STORE

In [22]:
df = pd.concat([bv, pv])
df['Customer Name'] = df['Customer Name']+' ('+df['Customer Code'].astype(str)+')'

In [23]:
print(df.shape)
df.head(2)

(4321, 8)


Bill No  Customer Name  Customer Code                Date  Bill Total  \
0      1.0  ABHAYRAJ (59)             59 2021-11-22 16:53:00      1861.0   
1      2.0  ABHAYRAJ (59)             59 2021-11-29 10:20:00       398.0   

  Payment  Amount Paid Received By  
0  CREDIT          NaN         NaN  
1  CREDIT          NaN         NaN

In [24]:
df = df[['Customer Name', 'Date', 'Bill Total', 'Amount Paid', 'Payment']]
dfa = df.groupby(['Customer Name'])[['Bill Total', 'Amount Paid']].agg('sum').reset_index()

In [25]:
print(df.shape)
df.head(2)

(4321, 5)


Customer Name                Date  Bill Total  Amount Paid Payment
0  ABHAYRAJ (59) 2021-11-22 16:53:00      1861.0          NaN  CREDIT
1  ABHAYRAJ (59) 2021-11-29 10:20:00       398.0          NaN  CREDIT

In [26]:
print(dfa.shape)
dfa.head(2)

(123, 3)


Customer Name  Bill Total  Amount Paid
0        ABHAYRAJ (59)      7273.0       5776.0
1  ABHIJIT PANDEY (15)       538.0        538.0

In [27]:
def days_between(d1, d2):
    d1 = datetime.strptime(d1, "%Y-%m-%d")
    d2 = datetime.strptime(d2, "%Y-%m-%d")
    return str(abs((d2 - d1).days))

In [28]:
def days_pending(custname, df):

    bal = df[(df['Payment'] == 'CREDIT') & (df['Customer Name'] == custname)]
    pay = df[(df['Payment'].isnull()) & (df['Customer Name'] == custname)]
    
    if len(pay.index) > 0:
        DLP = pay['Date'].max().strftime('%Y-%m-%d')
    elif len(bal.index) > 0:
        DLP = bal['Date'].min().strftime('%Y-%m-%d')

    if len(pay.index) > 0 or len(bal.index) > 0:
        TDY = datetime.today().strftime('%Y-%m-%d')
        DTE = days_between(DLP, TDY)
    else:
        DTE = str(0)
        
    return str(DTE)

In [30]:
dfa['Days Pending'] = dfa['Customer Name'].apply(lambda x: days_pending(x, df))
dfa['Balance Amount'] = dfa['Bill Total'] - dfa['Amount Paid']
dfa.sort_values(by=['Balance Amount', 'Days Pending'], inplace=True, ascending=False)

In [31]:
dfa.head()

Customer Name  Bill Total  Amount Paid Days Pending  Balance Amount
79  RAJESH MANJHI (28)     44305.0      22700.0           27         21605.0
55           MERAJ (3)     20445.0          0.0          136         20445.0
39    GUDDU MAHTO (19)     29704.0      10000.0           43         19704.0
95     ROHIT KALU (22)     19072.0       4000.0           65         15072.0
44     KAMLESH SAH (2)     14747.0          0.0          129         14747.0

### KhataBook

In [1]:
import pandas as pd
from datetime import datetime
import numpy as np

In [2]:
Path = "D:\\Projects\\mystoreapp\\"

In [3]:
bv = pd.read_csv(Path+"data/CreditDB.csv", parse_dates=['Date']).dropna()
pv = pd.read_csv(Path+"data/Payments.csv", parse_dates=['Date']).dropna()

In [4]:
df = pd.concat([bv, pv])
df['Customer Name'] = df['Customer Name']+' ('+df['Customer Code'].astype(str)+')'

In [5]:
df = df[['Customer Name', 'Date', 'Bill Total', 'Amount Paid', 'Payment']]
df = df.sort_values(by=['Customer Name', 'Date']).reset_index(drop=True)

In [6]:
# df = df[df['Customer Name'] == 'ASHOK (39)']
df.head(15)

Customer Name                Date  Bill Total  Amount Paid Payment
0   ABHAYRAJ (59) 2021-11-22 16:53:00      1861.0          NaN  CREDIT
1   ABHAYRAJ (59) 2021-11-29 10:20:00       398.0          NaN  CREDIT
2   ABHAYRAJ (59) 2021-12-04 21:20:00       420.0          NaN  CREDIT
3   ABHAYRAJ (59) 2021-12-07 19:17:00       325.0          NaN  CREDIT
4   ABHAYRAJ (59) 2021-12-15 08:32:00       233.0          NaN  CREDIT
5   ABHAYRAJ (59) 2021-12-15 10:58:00         NaN       3376.0     NaN
6   ABHAYRAJ (59) 2021-12-21 22:49:00       460.0          NaN  CREDIT
7   ABHAYRAJ (59) 2021-12-28 21:37:00       261.0          NaN  CREDIT
8   ABHAYRAJ (59) 2022-01-01 23:43:00       553.0          NaN  CREDIT
9   ABHAYRAJ (59) 2022-01-02 21:32:00       275.0          NaN  CREDIT
10  ABHAYRAJ (59) 2022-01-04 20:53:00       632.0          NaN  CREDIT
11  ABHAYRAJ (59) 2022-01-12 23:37:00       362.0          NaN  CREDIT
12  ABHAYRAJ (59) 2022-01-15 19:21:00         NaN       2400.0     NaN
13  ABHAYRAJ (59) 2022-01-17 21:59:00       830.0          NaN  CREDIT
14  ABHAYRAJ (59) 2022-02-10 23:06:00       663.0          NaN  CREDIT

In [7]:
df = df[df['Customer Name'] == 'ABHAYRAJ (59)']
df['Group'] = '.'

In [8]:
print(df)

    Customer Name                Date  Bill Total  Amount Paid Payment Group
0   ABHAYRAJ (59) 2021-11-22 16:53:00      1861.0          NaN  CREDIT     .
1   ABHAYRAJ (59) 2021-11-29 10:20:00       398.0          NaN  CREDIT     .
2   ABHAYRAJ (59) 2021-12-04 21:20:00       420.0          NaN  CREDIT     .
3   ABHAYRAJ (59) 2021-12-07 19:17:00       325.0          NaN  CREDIT     .
4   ABHAYRAJ (59) 2021-12-15 08:32:00       233.0          NaN  CREDIT     .
5   ABHAYRAJ (59) 2021-12-15 10:58:00         NaN       3376.0     NaN     .
6   ABHAYRAJ (59) 2021-12-21 22:49:00       460.0          NaN  CREDIT     .
7   ABHAYRAJ (59) 2021-12-28 21:37:00       261.0          NaN  CREDIT     .
8   ABHAYRAJ (59) 2022-01-01 23:43:00       553.0          NaN  CREDIT     .
9   ABHAYRAJ (59) 2022-01-02 21:32:00       275.0          NaN  CREDIT     .
10  ABHAYRAJ (59) 2022-01-04 20:53:00       632.0          NaN  CREDIT     .
11  ABHAYRAJ (59) 2022-01-12 23:37:00       362.0          NaN  CREDIT     .

In [9]:
i = 1

for ix, row in df.iterrows():
    if ix == 0:
        df.at[ix, 'Group'] = i
        
    if ix > 0:
        preval = df['Payment'][ix-1]
        curval = df['Payment'][ix]
        
        if preval == curval:
            df.at[ix, 'Group'] = i

        elif preval != curval:
            df.at[ix, 'Group'] = i+1

            i += 1    

In [10]:
print(df)
dfa = df.groupby(['Customer Name', 'Group'])[['Bill Total', 'Amount Paid']].agg('sum').reset_index()
dft = df.groupby('Group').agg(From=('Date', np.min), To=('Date', np.max)).reset_index()

    Customer Name                Date  Bill Total  Amount Paid Payment Group
0   ABHAYRAJ (59) 2021-11-22 16:53:00      1861.0          NaN  CREDIT     1
1   ABHAYRAJ (59) 2021-11-29 10:20:00       398.0          NaN  CREDIT     1
2   ABHAYRAJ (59) 2021-12-04 21:20:00       420.0          NaN  CREDIT     1
3   ABHAYRAJ (59) 2021-12-07 19:17:00       325.0          NaN  CREDIT     1
4   ABHAYRAJ (59) 2021-12-15 08:32:00       233.0          NaN  CREDIT     1
5   ABHAYRAJ (59) 2021-12-15 10:58:00         NaN       3376.0     NaN     2
6   ABHAYRAJ (59) 2021-12-21 22:49:00       460.0          NaN  CREDIT     3
7   ABHAYRAJ (59) 2021-12-28 21:37:00       261.0          NaN  CREDIT     3
8   ABHAYRAJ (59) 2022-01-01 23:43:00       553.0          NaN  CREDIT     3
9   ABHAYRAJ (59) 2022-01-02 21:32:00       275.0          NaN  CREDIT     3
10  ABHAYRAJ (59) 2022-01-04 20:53:00       632.0          NaN  CREDIT     3
11  ABHAYRAJ (59) 2022-01-12 23:37:00       362.0          NaN  CREDIT     3

In [14]:
print(dfa)

   Customer Name  Group  Bill Total  Amount Paid                From  \
0  ABHAYRAJ (59)      1      3237.0          0.0 2021-11-22 16:53:00   
1  ABHAYRAJ (59)      2         0.0       3376.0 2021-12-15 10:58:00   
2  ABHAYRAJ (59)      3      2543.0          0.0 2021-12-21 22:49:00   
3  ABHAYRAJ (59)      4         0.0       2400.0 2022-01-15 19:21:00   
4  ABHAYRAJ (59)      5      1493.0          0.0 2022-01-17 21:59:00   

                   To  
0 2021-12-15 08:32:00  
1 2021-12-15 10:58:00  
2 2022-01-12 23:37:00  
3 2022-01-15 19:21:00  
4 2022-02-10 23:06:00  


In [12]:
print(dft)

   Group                From                  To
0      1 2021-11-22 16:53:00 2021-12-15 08:32:00
1      2 2021-12-15 10:58:00 2021-12-15 10:58:00
2      3 2021-12-21 22:49:00 2022-01-12 23:37:00
3      4 2022-01-15 19:21:00 2022-01-15 19:21:00
4      5 2022-01-17 21:59:00 2022-02-10 23:06:00


In [13]:
dfa = dfa.merge(dft, on='Group')